<a href="https://colab.research.google.com/github/RicoStaedeli/NLP2025_CQG/blob/main/Training/3_Training_1_SFT_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SFT Training to generate critical questions
In this training we try to use the generated scores for the evaluated SocratiQ dataset.

## Setup
First we define some constant values and also install all needed libraries



### Installation

In [1]:
!pip install --no-deps xformers triton unsloth_zoo
!pip install sentencepiece protobuf huggingface_hub hf_transfer
!pip install --no-deps unsloth
!pip install -U transformers
!pip install -U datasets
!pip install -U accelerate
!pip install -U peft
!pip install -U trl
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.1/138.1 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.6/265.6 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 22.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.5

In [2]:
from unsloth import FastLanguageModel
import shutil
import os
import torch
from datasets import load_dataset
import logging
from trl import SFTTrainer
from transformers import TrainingArguments, EarlyStoppingCallback, IntervalStrategy, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.7.0+cu126 with CUDA 1206 (you have 2.6.0+cu124)
    Python  3.11.12 (you have 3.11.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!


### Colab
This part is only relevant when using the notebook in google colab

In [3]:
from google.colab import userdata, drive

In [4]:
drive.mount('/content/drive')
token = userdata.get('GITHUB')

Mounted at /content/drive


Clone GitHub Repository to directly push generated files

In [5]:
repo_url = f"https://{token}@github.com/RicoStaedeli/NLP2025_CQG.git"

!git clone {repo_url}

Cloning into 'NLP2025_CQG'...
remote: Enumerating objects: 870, done.
remote: Counting objects: 100% (133/133), done.
remote: Compressing objects: 100% (91/91), done.
remote: Total 870 (delta 87), reused 52 (delta 42), pack-reused 737 (from 2)
Receiving objects: 100% (870/870), 25.78 MiB | 16.93 MiB/s, done.
Resolving deltas: 100% (439/439), done.


### Path Variables and Logger

In [6]:
################################################################################
#######################   STATIC VARIABLES      ################################
################################################################################

TRAINING_NUMBER = 2
BASE_MODEL_REPO = "unsloth/Llama-3.1-8B-Instruct"
MODEL_NAME = "Meta-Llama-3.1-1B-Instruct_SFT_2"

################################################################################
#######################   PATH VARIABLES        ################################
################################################################################

train_dataset_path = "/content/drive/MyDrive/HSG/NLP/Project NLP/Data/Final/scored/processed_train_data.json" #"/content/NLP2025_CQG/Data/Processed/example_train.json"

log_base_path = f"/content/NLP2025_CQG/Training/Logs/Traing_{TRAINING_NUMBER}/Tensorboard/"
os.makedirs(log_base_path, exist_ok=True)

log_file_path = f"/content/NLP2025_CQG/Logs/training_{TRAINING_NUMBER}.log"

model_save_path = f"/content/drive/MyDrive/HSG/NLP/Project NLP/Training/Training_{TRAINING_NUMBER}/Model/{MODEL_NAME}_finetuned/"
os.makedirs(model_save_path, exist_ok=True)

model_lora_adapter_save_path = f"/content/drive/MyDrive/HSG/NLP/Project NLP/Training/Training_{TRAINING_NUMBER}/Model/{MODEL_NAME}_lora_adapters/"
os.makedirs(model_lora_adapter_save_path, exist_ok=True)


checkpoint_dir = f"/content/drive/MyDrive/HSG/NLP/Project NLP/Training/Training_{TRAINING_NUMBER}/Checkpoints/"
os.makedirs(checkpoint_dir, exist_ok=True)


################################################################################
#######################   LOGGER                ################################
################################################################################

# Setup logger manually
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# Create file handler (only if not already added)
if not logger.handlers:
    fh = logging.FileHandler(log_file_path)
    fh.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    fh.setFormatter(formatter)
    logger.addHandler(fh)

# Detect device
device = torch.device(
    "mps" if torch.backends.mps.is_available()
    else "cuda" if torch.cuda.is_available()
    else "cpu"
)

In [7]:
logger.info("--------  Start with Training  -------------")
logger.info(f'Device selected: {device}')
logger.info(f'Model: {MODEL_NAME}')
logger.info(f'Training number: {TRAINING_NUMBER}')

INFO:__main__:--------  Start with Training  -------------
INFO:__main__:Device selected: cuda
INFO:__main__:Model: Meta-Llama-3.1-1B-Instruct_SFT_2
INFO:__main__:Training number: 2


## Training Parameters

In [8]:
################################################################################
#######################   Unlsoth Parameters    ################################
################################################################################

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


################################################################################
#######################   PEFT Parameters       ################################
################################################################################

r = 16 # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                  "gate_proj", "up_proj", "down_proj",]
lora_alpha = 16
lora_dropout = 0 # Supports any, but = 0 is optimized
bias = "none"    # Supports any, but = "none" is optimized
use_gradient_checkpointing = "unsloth" # True or "unsloth" for very long context
random_state = 3407
use_rslora = False  # Unsloth supports rank stabilized LoRA
loftq_config = None # And LoftQ


################################################################################
#######################   SFT Trainer Parameters   #############################
################################################################################

dataset_text_field = "text"
dataset_num_proc = 2
packing = False
per_device_train_batch_size = 16
gradient_accumulation_steps = 4
warmup_steps = 5
num_train_epochs = 1
max_steps = 30
learning_rate = 2e-4
fp16 = not is_bfloat16_supported()
bf16 = is_bfloat16_supported()
logging_steps = 1
save_strategy = IntervalStrategy.STEPS
save_steps = 10
save_total_limit = 1
optim = "adamw_8bit"
weight_decay = 0.01
lr_scheduler_type = "linear"
seed = 3407
output_dir = checkpoint_dir
report_to = "tensorboard"
logging_dir = log_base_path

################################################################################
#######################   Log Parameters            ############################
################################################################################

logger.info("------ Unlsoth Parameters ---------------")
logger.info(f"max_seq_length: {max_seq_length}")
logger.info(f"dtype: {dtype}")
logger.info(f"load_in_4bit: {load_in_4bit}")

logger.info("------ PEFT Parameters ------------------")
logger.info(f"r: {r}")
logger.info(f"target_modules: {target_modules}")
logger.info(f"lora_alpha: {lora_alpha}")
logger.info(f"lora_dropout: {lora_dropout}")
logger.info(f"bias: {bias}")
logger.info(f"use_gradient_checkpointing: {use_gradient_checkpointing}")
logger.info(f"random_state: {random_state}")
logger.info(f"use_rslora: {use_rslora}")

logger.info("------  SFT Trainer Parameters ----------")
logger.info(f"dataset_text_field: {dataset_text_field}")
logger.info(f"dataset_num_proc: {dataset_num_proc}")
logger.info(f"packing: {packing}")
logger.info(f"per_device_train_batch_size: {per_device_train_batch_size}")
logger.info(f"gradient_accumulation_steps: {gradient_accumulation_steps}")
logger.info(f"warmup_steps: {warmup_steps}")
logger.info(f"max_steps: {max_steps}")
logger.info(f"learning_rate: {learning_rate}")
logger.info(f"fp16: {fp16}")
logger.info(f"bf16: {bf16}")
logger.info(f"logging_steps: {logging_steps}")
logger.info(f"save_strategy: {save_strategy}")
logger.info(f"save_steps: {save_steps}")
logger.info(f"save_total_limit: {save_total_limit}")
logger.info(f"optim: {optim}")
logger.info(f"weight_decay: {weight_decay}")
logger.info(f"lr_scheduler_type: {lr_scheduler_type}")
logger.info(f"seed: {seed}")
logger.info(f"output_dir: {output_dir}")
logger.info(f"report_to: {report_to}")
logger.info(f"logging_dir: {logging_dir}")

INFO:__main__:------ Unlsoth Parameters ---------------
INFO:__main__:max_seq_length: 2048
INFO:__main__:dtype: None
INFO:__main__:load_in_4bit: True
INFO:__main__:------ PEFT Parameters ------------------
INFO:__main__:r: 16
INFO:__main__:target_modules: ['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj']
INFO:__main__:lora_alpha: 16
INFO:__main__:lora_dropout: 0
INFO:__main__:bias: none
INFO:__main__:use_gradient_checkpointing: unsloth
INFO:__main__:random_state: 3407
INFO:__main__:use_rslora: False
INFO:__main__:------  SFT Trainer Parameters ----------
INFO:__main__:dataset_text_field: text
INFO:__main__:dataset_num_proc: 2
INFO:__main__:packing: False
INFO:__main__:per_device_train_batch_size: 2
INFO:__main__:gradient_accumulation_steps: 4
INFO:__main__:warmup_steps: 5
INFO:__main__:max_steps: 30
INFO:__main__:learning_rate: 0.0002
INFO:__main__:fp16: False
INFO:__main__:bf16: True
INFO:__main__:logging_steps: 1
INFO:__main__:save_strategy: IntervalStrate

### Unsloth

In [9]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = BASE_MODEL_REPO,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.5.6: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [10]:
model = FastLanguageModel.get_peft_model(
    model,
    r = r,
    target_modules = target_modules,
    lora_alpha = lora_alpha,
    lora_dropout = lora_dropout,
    bias = bias,
    use_gradient_checkpointing = use_gradient_checkpointing,
    random_state = random_state,
    use_rslora = use_rslora,
    loftq_config = loftq_config,
)

Unsloth 2025.5.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## Load and preprocess dataset
The raw dataset [SocratiQ](https://github.com/NUS-IDS/eacl23_soqg/tree/main) has a label at the begining of the context. We have to remove that and also tokenize the input for the model training.

In [11]:
dataset = load_dataset('json', data_files=train_dataset_path)

Generating train split: 0 examples [00:00, ? examples/s]

In [12]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'question', 'context_token_len', 'is_Critical', 'CauseToEffect', 'Analogy', 'ExpertOpinion', 'FearAppeal'],
        num_rows: 80985
    })
})


In [13]:
# crop the dataset for testing
#dataset['train'] = dataset['train'].select(range(500))

In [16]:
base_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Generate one critical question addressing the given context following four different schema types. Your answer is just the question without anything else.

Schmea types are:
CauseToEffect:
How strong is the generalisation that if <eventA> then <eventB>?
Are there other factors in this particular case that could have interfered with the event of‘<eventB>’?

ExpertOpinion:
Is <expertE> a genuine expert in <domainD>?
Is <eventA> consistent with what other experts in <domainD> say?

Analogy:
Are <C1> and <C2> similar in the respect cited?
Is <eventA> true in <C1>?

FearAppeal:
Is <eventB> bad? Why and to whom is it bad?
Is <eventA> away to prevent <eventB>?



This is the given context to relate the question to:

### Context:
{}

### Response:
{}

### Evaluation:
This Response scored with the following score:
CauseToEffect: {}
Analogy: {}
ExpertOpinion: {}
FearAppeal: {}

"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    contexts       = examples["context"]
    questions      = examples["question"]
    CauseToEffect         = examples["CauseToEffect"]
    Analogy               = examples["Analogy"]
    ExpertOpinion         = examples["ExpertOpinion"]
    FearAppeal            = examples["FearAppeal"]
    texts = []
    for input, output, CauseToEffect, Analogy, ExpertOpinion, FearAppeal in zip(contexts, questions, CauseToEffect, Analogy, ExpertOpinion, FearAppeal):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = base_prompt.format(input, output, CauseToEffect, Analogy, ExpertOpinion, FearAppeal) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/80985 [00:00<?, ? examples/s]

In [17]:
# Check the structure of the dataset
print(dataset['train'][0]['text'])

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Generate one critical question addressing the given context following four different schema types. Your answer is just the question without anything else.

Schmea types are:
CauseToEffect: 
How strong is the generalisation that if <eventA> then <eventB>?
Are there other factors in this particular case that could have interfered with the event of‘<eventB>’?

ExpertOpinion: 
Is <expertE> a genuine expert in <domainD>?
Is <eventA> consistent with what other experts in <domainD> say?

Analogy:
Are <C1> and <C2> similar in the respect cited?
Is <eventA> true in <C1>?

FearAppeal:
Is <eventB> bad? Why and to whom is it bad?
Is <eventA> away to prevent <eventB>?



This is the given context to relate the question to:

### Context:
alternate_viewpoints_perspectives: A parallel argument would state that England is worse o

In [18]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'question', 'context_token_len', 'is_Critical', 'CauseToEffect', 'Analogy', 'ExpertOpinion', 'FearAppeal', 'text'],
        num_rows: 80985
    })
})


In [19]:
train_dataset = dataset['train']

In [20]:
print(train_dataset)

Dataset({
    features: ['id', 'context', 'question', 'context_token_len', 'is_Critical', 'CauseToEffect', 'Analogy', 'ExpertOpinion', 'FearAppeal', 'text'],
    num_rows: 80985
})


### Train the model

In [21]:
args = TrainingArguments(
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    warmup_steps=warmup_steps,
    # max_steps=max_steps,
    num_train_epochs=num_train_epochs,
    learning_rate=learning_rate,
    fp16=fp16,
    bf16=bf16,
    logging_steps=logging_steps,
    save_strategy=save_strategy,
    save_steps=save_steps,
    save_total_limit=save_total_limit,
    optim=optim,
    weight_decay=weight_decay,
    lr_scheduler_type=lr_scheduler_type,
    seed=seed,
    output_dir=output_dir,
    report_to=report_to,
    logging_dir=logging_dir,
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = dataset_text_field,
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = dataset_num_proc,
    packing = packing,
    args = args,
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/80985 [00:00<?, ? examples/s]

In [22]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)

logger.info(f"GPU  Information before Training")
logger.info(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
logger.info(f"{start_gpu_memory} GB of memory reserved.")

INFO:__main__:GPU  Information before Training
INFO:__main__:GPU = NVIDIA L4. Max memory = 22.161 GB.
INFO:__main__:7.623 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 80,985 | Num Epochs = 1 | Total steps = 10,123
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,0.533500
2,0.568100
3,0.520200
4,0.345300
5,0.086900
6,0.035700
7,0.025600
8,0.023700
9,0.021500
10,0.020900


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)

logger.info(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
logger.info(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
logger.info(f"Peak reserved memory = {used_memory} GB.")
logger.info(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
logger.info(f"Peak reserved memory % of max memory = {used_percentage} %.")
logger.info(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

## Save
Save the finetuned model

### Save Lora delta weights

In [ ]:
model.save_pretrained(model_lora_adapter_save_path)  # Local saving
tokenizer.save_pretrained(model_lora_adapter_save_path)
logger.info(f"Saved LoRA adapters to {model_lora_adapter_save_path}")

### Saving merged model

Save merged model

In [ ]:
# Merge to 16bit
if False:
  model.save_pretrained_merged(model_save_path, tokenizer, save_method = "merged_16bit",)
  logger.info(f"Saved merged model in 16bit to {model_save_path}")
if True:
  token = userdata.get('HF_TOKEN2')
  model.push_to_hub_merged(f"ricostaedeli/{MODEL_NAME}", tokenizer, save_method="merged_16bit", token=token, private=True)

# Merge to 4bit
if False:
  model.save_pretrained_merged(model_save_path, tokenizer, save_method = "merged_4bit",)
  logger.info(f"Saved merged model in 4bit to {model_save_path}")

# Just LoRA adapters
if False:
  model.save_pretrained_merged(model_save_path, tokenizer, save_method = "lora",)
  logger.info(f"Saved LoRA adapters to {model_save_path}")

### GGUF / llama.cpp Conversion
Save model in GGUF format

Some supported quant methods (full list on [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

In [ ]:
# Save to 8bit Q8_0
if False:
  model.save_pretrained_gguf(model_save_path, tokenizer,)
  logger.info(f"Saved merged model in 8bit Q8_0 to {model_save_path}")

# Save to 16bit GGUF
if False:
  model.save_pretrained_gguf(model_save_path, tokenizer, quantization_method = "f16")
  logger.info(f"Saved merged model in 16bit GGUF to {model_save_path}")

# Save to q4_k_m GGUF
if False:
  model.save_pretrained_gguf(model_save_path, tokenizer, quantization_method = "q4_k_m")
  logger.info(f"Saved merged model in q4_k_m GGUF to {model_save_path}")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

In [ ]:
os.chdir("NLP2025_CQG")
!ls

In [ ]:
!git config --global user.name "Rico Städeli"
!git config --global user.email "rico@yabriga.ch"


commit_message = f"Training Number: {TRAINING_NUMBER}, Training logs in Google Drive."
!git add .
!git commit -m "{commit_message}"
!git push